# nb_runtype: Test Suite

This notebook provides a thorough, CI-friendly test suite for the `nb_runtype` library. It demonstrates and validates all features, including:

- Standard Python type annotation behavior (no runtime enforcement)
- Automatic runtime validation with `nb_runtype`
- Configuration options and their effects
- Per-function exclusion and disabling
- Validation of default values, arbitrary types, and coercion
- Async function support
- Robust assertions for automated testing


In [1]:
# Import the library and set up environment
import os
import sys

# Add repo root to sys.path for CI and local runs
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

## Standard Python Type Annotations: No Runtime Enforcement

This section demonstrates that Python's type annotations are not enforced at runtime by default. Functions can be called with arguments or return values of any type, regardless of their annotations.


In [2]:
# Standard Python type annotations: no runtime enforcement
def py_mul(x: int, y: int) -> int:
    return x * y


# No error for wrong return type (should return int, but returns str if misused)
result = py_mul(2, 3)
print("py_mul(2, 3) result:", result)
assert result == 6, f"Expected 6, got {result}"

# No error for wrong argument type (should be int, but accepts str)
result2 = py_mul("a", 2)
print("py_mul('a', 2) result:", result2)
assert result2 == "aa", f"Expected 'aa', got {result2}"

py_mul(2, 3) result: 6
py_mul('a', 2) result: aa


## Enabling nb_runtype: Automatic Runtime Validation

Now we enable `nb_runtype` to automatically validate all new function definitions at runtime.


In [3]:
from nb_runtype import (
    RuntypeError,
    disable_runtype,
    enable_runtype,
    get_runtype_config,
    is_runtype_enabled,
    no_runtype,
)

## Functions Defined Before and After Enabling nb_runtype

Functions defined before calling `enable_runtype()` (or in the same cell) are not validated at runtime, even if called after enabling. Only functions defined in subsequent cells are validated.


In [4]:
# Function defined before enabling nb_runtype: not validated
def pre_enable(x: int) -> int:
    return str(x)


# Enable nb_runtype
enable_runtype()


# Function defined after enabling: will be validated
def post_enable(x: int) -> int:
    return str(x)


# No error for function defined before enabling, even with wrong return type
assert pre_enable(2) == "2"
try:
    pre_enable("a")
except Exception as e:
    print("pre_enable('a') does not raise RuntypeError, but:", type(e), e)
else:
    print('pre_enable("a") does not raise any error (expected behavior)')

# No error for function defined in the same cell, even with wrong return type
assert post_enable(2) == "2"
try:
    post_enable("a")
except Exception as e:
    print("post_enable('a') does not raise RuntypeError, but:", type(e), e)
else:
    print('post_enable("a") does not raise any error (expected behavior)')

runtype enabled with config={'strict': True, 'validate_return': True, 'validate_default': True, 'arbitrary_types_allowed': True}
pre_enable("a") does not raise any error (expected behavior)
post_enable("a") does not raise any error (expected behavior)


In [5]:
# Function defined after enabling: validated by nb_runtype
def mul(x: int, y: int) -> int:
    return x * y


# Correct types: should work
result = mul(2, 3)
print("mul(2, 3) result:", result)
assert result == 6, f"Expected 6, got {result}"

# Wrong argument type: should raise RuntypeError
try:
    mul("a", 2)
except RuntypeError as e:
    print("RuntypeError for wrong argument type as expected:", e)
else:
    raise AssertionError("Expected RuntypeError for wrong argument type, but none was raised.")


# Wrong return type: should raise RuntypeError
def mul_str(x: int, y: int) -> int:
    return str(x * y)


try:
    mul_str(2, 3)
except RuntypeError as e:
    print("RuntypeError for wrong return type as expected:", e)
else:
    raise AssertionError("Expected RuntypeError for wrong return type, but none was raised.")

mul(2, 3) result: 6
RuntypeError for wrong argument type as expected: Runtime type validation failed:
  Parameter '0': Input should be a valid integer. Value: 'a'
RuntypeError for wrong return type as expected: Runtime type validation failed:
  Parameter 'return': Input should be a valid integer. Value: '6'


## Per-Function Exclusion: `@no_runtype` Decorator

Functions decorated with `@no_runtype` are excluded from runtime validation, even after enabling nb_runtype.


In [6]:
# Function excluded from validation using @no_runtype
@no_runtype
def no_type_check_mult(x: int, y: int) -> int:
    return str(x) * y


# Should not raise RuntypeError, even with wrong types or return type
result = no_type_check_mult("a", 2)
print("no_type_check_mult('a', 2) result:", result)
assert result == "aa", f"Expected 'aa', got {result}"
result2 = no_type_check_mult(2, 3)
print("no_type_check_mult(2, 3) result:", result2)
assert result2 == "222", f"Expected '222', got {result2}"

no_type_check_mult('a', 2) result: aa
no_type_check_mult(2, 3) result: 222


## Retrieving the Current nb_runtype Configuration

You can retrieve the current configuration to verify the active validation settings.


In [7]:
# Retrieve and check the current nb_runtype configuration
config = get_runtype_config()
print("Current config:", config)
assert config["strict"] is True
assert config["validate_return"] is True
assert config["validate_default"] is True
assert config["arbitrary_types_allowed"] is True

Current config: {'strict': True, 'validate_return': True, 'validate_default': True, 'arbitrary_types_allowed': True}


## Checking if nb_runtype is Enabled: `is_runtype_enabled`

The function `is_runtype_enabled()` returns True if runtime type validation is currently active, and False otherwise.


In [8]:
# Test is_runtype_enabled in both enabled and disabled states
disable_runtype()
assert is_runtype_enabled() is False, "Expected is_runtype_enabled() to be False after disabling runtype"
print("is_runtype_enabled() after disable:", is_runtype_enabled())

enable_runtype()
assert is_runtype_enabled() is True, "Expected is_runtype_enabled() to be True after enabling runtype"
print("is_runtype_enabled() after enable:", is_runtype_enabled())

runtype disabled.
is_runtype_enabled() after disable: False
runtype enabled with config={'strict': True, 'validate_return': True, 'validate_default': True, 'arbitrary_types_allowed': True}
is_runtype_enabled() after enable: True


## Disabling nb_runtype: Return to Standard Python Behavior

After disabling nb_runtype, new functions are not validated and behave like standard Python functions.


In [9]:
# Disable runtype
disable_runtype()

runtype disabled.


In [10]:
# Function defined after disabling nb_runtype: not validated
def py_mul_post_disable(x: int, y: int) -> int:
    return str(x) * int(y)


# No error for wrong return type
result = py_mul_post_disable(2, 3)
print("py_mul_post_disable(2, 3) result:", result)
assert result == "222", f"Expected '222', got {result}"

# No error for wrong argument type
result2 = py_mul_post_disable("a", 2)
print("py_mul_post_disable('a', 2) result:", result2)
assert result2 == "aa", f"Expected 'aa', got {result2}"

py_mul_post_disable(2, 3) result: 222
py_mul_post_disable('a', 2) result: aa


## Functions Without Type Annotations: No Validation

Functions without type annotations are never validated by nb_runtype, even if validation is enabled.


In [11]:
# Enable runtype
enable_runtype()

runtype enabled with config={'strict': True, 'validate_return': True, 'validate_default': True, 'arbitrary_types_allowed': True}


In [12]:
# Function without type annotations: not validated
def no_annot(x, y):
    return str(x) * int(y)


# No error even with 'wrong' types
result = no_annot("a", 2)
print("no_annot('a', 2) result:", result)
assert result == "aa", f"Expected 'aa', got {result}"
result2 = no_annot(2, 3)
print("no_annot(2, 3) result:", result2)
assert result2 == "222", f"Expected '222', got {result2}"

no_annot('a', 2) result: aa
no_annot(2, 3) result: 222


## Type Annotations with `Any`: No Restriction

If a function argument or return value is annotated as `Any`, nb_runtype allows any value for that parameter or return, without raising errors.


In [13]:
from typing import Any


# Function with Any annotations: accepts any type for arguments and return value
def any_args(x: Any, y: Any) -> Any:
    return x, y


# No error for any type of argument or return value
result = any_args(1, "a")
print('any_args(1, "a") result:', result)
assert result == (1, "a")
result2 = any_args([1, 2], {"k": 3})
print('any_args([1,2], {"k":3}) result:', result2)
assert result2 == ([1, 2], {"k": 3})
result3 = any_args(None, 3.14)
print("any_args(None, 3.14) result:", result3)
assert result3 == (None, 3.14)


# Return value can also be of any type
def any_return(x: int) -> Any:
    if x == 0:
        return "zero"
    return [x]


assert any_return(0) == "zero"
assert any_return(5) == [5]

any_args(1, "a") result: (1, 'a')
any_args([1,2], {"k":3}) result: ([1, 2], {'k': 3})
any_args(None, 3.14) result: (None, 3.14)


## Error Handling & Debugging

When type validation fails, nb-runtype raises a `RuntypeError` with detailed information about what went wrong. Understanding these errors is crucial for effective debugging.


In [14]:
enable_runtype()

runtype already enabled.


In [15]:
def divide_numbers(x: float, y: float) -> float:
    """Function that divides two numbers."""
    return x / y


# Valid call
result = divide_numbers(10.0, 2.0)
print(f"divide_numbers(10.0, 2.0) result:", result)
assert result == 5.0

# Type error in arguments
try:
    divide_numbers("10", 2.0)  # String instead of float
except RuntypeError as e:
    print(f"Argument error caught: {e}")
else:
    raise AssertionError("Expected RuntypeError for non-float argument, but none was raised.")


# Type error in return value
def bad_divide(x: float, y: float) -> float:
    return str(x / y)  # Returns string instead of float


try:
    bad_divide(10.0, 2.0)
except RuntypeError as e:
    print(f"Return error caught: {e}")
else:
    raise AssertionError("Expected RuntypeError for non-float return value, but none was raised.")

# Non-type errors are not caught
try:
    divide_numbers(10.0, 0.0)  # This will raise ZeroDivisionError
except ZeroDivisionError as e:
    print(f"Non-type error (not caught): {type(e).__name__}: {e}")
except RuntypeError as e:
    raise AssertionError(f"Unexpected RuntypeError caught: {e}")

divide_numbers(10.0, 2.0) result: 5.0
Argument error caught: Runtime type validation failed:
  Parameter '0': Input should be a valid number. Value: '10'
Return error caught: Runtime type validation failed:
  Parameter 'return': Input should be a valid number. Value: '5.0'
Non-type error (not caught): ZeroDivisionError: float division by zero


## Configuration Errors: Non-Boolean Parameters to `enable_runtype`

Passing non-boolean values to `enable_runtype` parameters should raise a `TypeError`.


In [16]:
# Disable runtype
disable_runtype()

runtype disabled.


In [17]:
# Passing non-boolean values to enable_runtype should raise TypeError
try:
    enable_runtype(strict="yes")
except TypeError as e:
    print("TypeError for non-boolean 'strict':", e)
else:
    raise AssertionError("Expected TypeError for non-bool strict, but none was raised.")

try:
    enable_runtype(validate_return=1)
except TypeError as e:
    print("TypeError for non-boolean 'validate_return':", e)
else:
    raise AssertionError("Expected TypeError for non-bool validate_return, but none was raised.")

TypeError for non-boolean 'strict': All parameters must be boolean values
TypeError for non-boolean 'validate_return': All parameters must be boolean values


## Async Functions: Argument and Return Validation

Async functions are validated just like regular functions. Both arguments and return values are checked according to the current configuration.


In [18]:
disable_runtype()
enable_runtype()

runtype is not enabled.
runtype enabled with config={'strict': True, 'validate_return': True, 'validate_default': True, 'arbitrary_types_allowed': True}


In [19]:
from typing import Any


# Async function to test argument and return validation
async def async_add(x: int, y: int) -> Any:
    return x + y


# Correct types: should work
result = await async_add(2, 3)
print("async_add(2, 3) result:", result)
assert result == 5

# Wrong argument type: should raise RuntypeError
try:
    await async_add("a", 2)
except RuntypeError as e:
    print("Caught RuntypeError for async function as expected:", e)
else:
    raise AssertionError("Expected RuntypeError for wrong argument type in async function, but none was raised.")

async_add(2, 3) result: 5
Caught RuntypeError for async function as expected: Runtime type validation failed:
  Parameter '0': Input should be a valid integer. Value: 'a'


## Functions with Default Values: Validation of Defaults

Default values must be valid according to their type annotations when the function is called and validation is enabled.
This means:

- If `validate_default=True` (default): Invalid defaults cause errors at call time
- If `validate_default=False`: Invalid defaults are allowed and ignored


In [20]:
disable_runtype()
enable_runtype(validate_default=True)

runtype disabled.
runtype enabled with config={'strict': True, 'validate_return': True, 'validate_default': True, 'arbitrary_types_allowed': True}


In [21]:
# validate_default=True (default): error if default is invalid
def f_default(x: int = "a"):
    return x


try:
    f_default()
except RuntypeError as e:
    print("RuntypeError for invalid default value:", e)
else:
    raise AssertionError("Expected RuntypeError for invalid default, but none was raised.")

RuntypeError for invalid default value: Runtime type validation failed:
  Parameter 'x': Input should be a valid integer. Value: 'a'


### Disabling Default Value Validation

If `validate_default=False`, invalid default values are not checked and are allowed.


In [22]:
disable_runtype()
enable_runtype(validate_default=False)

runtype disabled.
runtype enabled with config={'strict': True, 'validate_return': True, 'validate_default': False, 'arbitrary_types_allowed': True}


In [23]:
# validate_default=False: no error for invalid default value
def f_default2(x: int = "a"):
    return x


print("f_default2() result:", f_default2())
assert f_default2() == "a"

f_default2() result: a


## Custom Types: The `arbitrary_types_allowed` Parameter

By default, nb_runtype accepts custom classes and types you've defined. However, you can restrict this behavior.

- `arbitrary_types_allowed=True` (default): Accepts any type, including custom classes
- `arbitrary_types_allowed=False`: Only accepts built-in types and types Pydantic knows about


In [24]:
disable_runtype()
enable_runtype()

runtype disabled.
runtype enabled with config={'strict': True, 'validate_return': True, 'validate_default': True, 'arbitrary_types_allowed': True}


In [25]:
# arbitrary_types_allowed=True (default): accepts custom types
class MyCustom:
    pass


def f_custom(x: MyCustom) -> MyCustom:
    return x


obj = MyCustom()
assert f_custom(obj) is obj
print("f_custom(obj) passed")

f_custom(obj) passed


In [26]:
disable_runtype()
enable_runtype(arbitrary_types_allowed=False)

runtype disabled.
runtype enabled with config={'strict': True, 'validate_return': True, 'validate_default': True, 'arbitrary_types_allowed': False}


In [27]:
# arbitrary_types_allowed=False: error for custom types
def f_custom2(x: MyCustom) -> MyCustom:
    return x


try:
    f_custom2(obj)
except Exception as e:
    print("RuntypeError for custom type with arbitrary_types_allowed=False:", e)
else:
    raise AssertionError("Expected RuntypeError for custom type, but none was raised.")

RuntypeError for custom type with arbitrary_types_allowed=False: Runtime type validation failed:
  Failed to generate Pydantic schema. Value: None


## Strict Mode vs Lenient Mode: Type Coercion Control

The `strict` parameter controls whether Pydantic attempts automatic type conversion.

**Strict Mode (`strict=True`, default)**:

- **No coercion**: Values must exactly match their annotated types
- **Safer**: Prevents unexpected type conversions
- **Faster**: Less processing overhead

**Lenient Mode (`strict=False`)**:

- **Allows coercion**: Attempts to convert values (e.g., `"42"` → `42`, `3.14` → `3`)
- **More flexible**: Accepts "reasonable" conversions
- **Slower**: Additional processing for type conversion


In [28]:
disable_runtype()
enable_runtype(strict=False)

runtype disabled.
runtype enabled with config={'strict': False, 'validate_return': True, 'validate_default': True, 'arbitrary_types_allowed': True}


In [29]:
# strict=False: type coercion is allowed
def f_coerce(x: int) -> int:
    return x


# float accepted as int
assert f_coerce(2.0) == 2
print("f_coerce(2.0) passed")
# numeric string accepted as int
assert f_coerce("3") == 3
print("f_coerce('3') passed")
# non-numeric string: error
try:
    f_coerce("a")
except RuntypeError as e:
    print("RuntypeError for failed coercion:", e)
else:
    raise AssertionError("Expected RuntypeError for non-coercible value, but none was raised.")

f_coerce(2.0) passed
f_coerce('3') passed
RuntypeError for failed coercion: Runtime type validation failed:
  Parameter '0': Input should be a valid integer, unable to parse string as an integer. Value: 'a'


## Multiple Decoration: No Double Wrapping

Redefining a function should not apply validation twice. Validation remains active, but the function is not double-wrapped.


In [30]:
# Setup for multiple decoration test
disable_runtype()
enable_runtype()

runtype disabled.
runtype enabled with config={'strict': True, 'validate_return': True, 'validate_default': True, 'arbitrary_types_allowed': True}


In [31]:
# Redefining a validated function: should not double-wrap, validation remains active
def f(x: int) -> int:
    return x


# Redefine the function: should not error, validation remains active
f = f
assert f(2) == 2
try:
    f("a")
except RuntypeError as e:
    print("RuntypeError for multiple decoration:", e)
else:
    raise AssertionError("Expected RuntypeError for wrong type, but none was raised.")

RuntypeError for multiple decoration: Runtime type validation failed:
  Parameter '0': Input should be a valid integer. Value: 'a'
